<a href="https://colab.research.google.com/github/JoaquinV13/UFV-Visualizacion/blob/main/Clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install plotly_express

In [ ]:
import plotly_express as px
import pandas as pd
import numpy as np
import re
import plotly.graph_objects as go
from google.colab import drive
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
drive.mount('/content/drive/')
data = pd.read_csv('/content/drive/MyDrive/Colab/datos_ejercicio_ventas.csv',sep=',')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Visualizamos los datos.
data.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


In [ ]:
# Tipos de datos.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18666 entries, 0 to 18665
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   COUNTRY        18666 non-null  object 
 1   SUBBRAND       18666 non-null  object 
 2   YEAR           18666 non-null  int64  
 3   MONTH          18666 non-null  int64  
 4   SCENARIO       18666 non-null  object 
 5   FORECAST       17766 non-null  object 
 6   FORECAST_YEAR  17766 non-null  float64
 7   AMOUNT         18666 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 1.1+ MB


In [ ]:
# Número de 'Actuals' y 'AI_forecast'.
actuals = (sum(data['SCENARIO']=='actual'))
forecast = (sum(data['SCENARIO']=='AI_forecast'))
print('Número de Actuals:',actuals)
print('Número de Forecasts:',forecast)

Número de Actuals: 900
Número de Forecasts: 17766


In [ ]:
# Número de regustros de cada marca por país.
tabla_pivot = data.pivot_table(index='COUNTRY', columns='SUBBRAND', aggfunc='size', fill_value=0)

# Agregar una columna con el total de registros por país.
tabla_pivot['Total'] = tabla_pivot.sum(axis=1)
print(tabla_pivot)

SUBBRAND       7up (L3)  7up Free (L3)  Lipton (L3)  Mountain Dew (L3)  \
COUNTRY                                                                  
Czech               236              7          236                236   
Denmark             216            236            0                236   
Great Britain       794            758          830                542   
Hungary             268            236          452                236   
Italy               236              0            0                  0   
Netherlands         236            236          452                216   
Norway              236            236          252                236   
Portugal            236            236          235                  0   
Spain               236            398          236                  0   

SUBBRAND       Pepsi Max (L3)  Pepsi Regular (L3)  Total  
COUNTRY                                                   
Czech                     236                 452   1403  
Denmark 

In [ ]:
# Todos los datos del dataset tiene el mismo horizonte de previsión. Calculamos
# el de uno, sabemos el de todos.
data_1 = data[(data['FORECAST_YEAR']==2023) & (data['FORECAST']=='AI_P02F') & (data['COUNTRY']=='Portugal') & (data['SCENARIO']=='AI_forecast') & (data['SUBBRAND']=='Lipton (L3)')]
print(len(data_1))

18


In [ ]:
# Comprobamos la cantidad de forecasts distintos que hay. Contando las diferentes
# fechas de inicio de un forecast.

data[(data['SCENARIO']=='AI_forecast')].loc[:, ['FORECAST_YEAR', 'FORECAST']].drop_duplicates().shape[0]

12

Vemos que la cantidad de forecast distintos es 12 pero, ¿esto es igual para todos los productos de cada país?

In [ ]:
# Comprobamos que para cada producto de cada país se han hecho 12 estudios
# predicitivos teniendo en cuenta la fecha de inicio de la predicción.

Pais_producto = data[(data['SCENARIO']=='AI_forecast')].groupby(['COUNTRY','SUBBRAND'])

# Contar los valores únicos de la combinación de 'Year' y 'caca'
resultado = Pais_producto.apply(lambda x: x[['FORECAST_YEAR', 'FORECAST']].drop_duplicates().shape[0], include_groups=False).reset_index(name='Estudios')

print(resultado)

          COUNTRY            SUBBRAND  Estudios
0           Czech            7up (L3)        12
1           Czech         Lipton (L3)        12
2           Czech   Mountain Dew (L3)        12
3           Czech      Pepsi Max (L3)        12
4           Czech  Pepsi Regular (L3)        12
5         Denmark            7up (L3)        12
6         Denmark       7up Free (L3)        12
7         Denmark   Mountain Dew (L3)        12
8         Denmark      Pepsi Max (L3)        12
9         Denmark  Pepsi Regular (L3)        12
10  Great Britain            7up (L3)        12
11  Great Britain       7up Free (L3)        12
12  Great Britain         Lipton (L3)        12
13  Great Britain   Mountain Dew (L3)        12
14  Great Britain      Pepsi Max (L3)        12
15  Great Britain  Pepsi Regular (L3)        12
16        Hungary            7up (L3)        12
17        Hungary       7up Free (L3)        12
18        Hungary         Lipton (L3)        12
19        Hungary   Mountain Dew (L3)   

Efectivamente, para cada producto de cada país se han hecho 12 estudios de predicción distintos.

In [ ]:
# Comprobamos que el amount tenga números con sentido.
data['AMOUNT'].describe()

,AMOUNT
count,1.866600e+04
mean,9.721822e+05
std,1.915283e+06
min,-2.171201e+05
25%,8.754541e+04
50%,3.081759e+05
75%,1.078576e+06
max,1.481563e+07


In [ ]:
# Hay números negativos, los eliminamos.
data = data[data['AMOUNT'] >= 0]

In [ ]:
# Sumamos la cantidad de cada producto del país.
datos = data[(data['SCENARIO'] == 'actual')].groupby(['COUNTRY', 'SUBBRAND'])['AMOUNT'].sum().reset_index()
print(datos)

          COUNTRY            SUBBRAND        AMOUNT
0           Czech            7up (L3)  9.610990e+05
1           Czech       7up Free (L3)  3.242474e+05
2           Czech         Lipton (L3)  5.959059e+06
3           Czech   Mountain Dew (L3)  1.604077e+06
4           Czech      Pepsi Max (L3)  7.922810e+06
5           Czech  Pepsi Regular (L3)  1.858035e+07
6         Denmark       7up Free (L3)  6.356360e+05
7         Denmark   Mountain Dew (L3)  8.253904e+05
8         Denmark      Pepsi Max (L3)  4.936140e+07
9         Denmark  Pepsi Regular (L3)  5.774250e+06
10  Great Britain            7up (L3)  5.432014e+06
11  Great Britain       7up Free (L3)  2.451862e+07
12  Great Britain         Lipton (L3)  1.365866e+07
13  Great Britain   Mountain Dew (L3)  3.692249e+06
14  Great Britain      Pepsi Max (L3)  2.496323e+08
15  Great Britain  Pepsi Regular (L3)  3.784479e+07
16        Hungary            7up (L3)  1.814551e+05
17        Hungary       7up Free (L3)  1.244282e+06
18        Hu

In [ ]:
# Normalizamos las cantidades de los productos diviendo entre el total de
# de cantidad vendida en el país.
country_totals = datos.groupby('COUNTRY')['AMOUNT'].sum().reset_index()
country_totals = country_totals.rename(columns={'AMOUNT': 'COUNTRY_TOTAL'})

datos_N = pd.merge(datos, country_totals, on='COUNTRY', how='left')
datos_N['AMOUNT_N'] = datos_N['AMOUNT'] / datos_N['COUNTRY_TOTAL']

# Eliminamos columnas innecesarias.
datos_N.drop(columns=['AMOUNT', 'COUNTRY_TOTAL'], inplace=True)

print(datos_N)

          COUNTRY            SUBBRAND  AMOUNT_N
0           Czech            7up (L3)  0.027187
1           Czech       7up Free (L3)  0.009172
2           Czech         Lipton (L3)  0.168565
3           Czech   Mountain Dew (L3)  0.045375
4           Czech      Pepsi Max (L3)  0.224114
5           Czech  Pepsi Regular (L3)  0.525587
6         Denmark       7up Free (L3)  0.011231
7         Denmark   Mountain Dew (L3)  0.014584
8         Denmark      Pepsi Max (L3)  0.872161
9         Denmark  Pepsi Regular (L3)  0.102025
10  Great Britain            7up (L3)  0.016226
11  Great Britain       7up Free (L3)  0.073238
12  Great Britain         Lipton (L3)  0.040799
13  Great Britain   Mountain Dew (L3)  0.011029
14  Great Britain      Pepsi Max (L3)  0.745664
15  Great Britain  Pepsi Regular (L3)  0.113044
16        Hungary            7up (L3)  0.004364
17        Hungary       7up Free (L3)  0.029925
18        Hungary         Lipton (L3)  0.157885
19        Hungary   Mountain Dew (L3)  0

# Distribución de ventas realizadas en cada país.

In [ ]:
fig = px.histogram(datos_N, x='COUNTRY', y='AMOUNT_N', color='SUBBRAND',
    title="Distribución de ventas realizadas por país",
    template="plotly_white",
    color_discrete_sequence=px.colors.qualitative.Prism, # Color pastel.
    height=700, # Modificamos la altura para que se vean bien las porporciones.
    width=1100)

fig.update_layout(xaxis_title="País",  yaxis_title="Proporción de las ventas",
    legend_title="Marca", title_font=dict(size=24, color="black", family="Arial"),
    font=dict(size=16, color="black", family="Arial"),
    bargap=0.4) # Ajustamos el espacio entre barras.

fig.show()

In [ ]:
# Total de ventas por país.
fig = px.histogram(country_totals, x='COUNTRY', y='COUNTRY_TOTAL', log_y=True,
                   color='COUNTRY',
                   template="plotly_white",
                   color_discrete_sequence=px.colors.qualitative.Prism,
                   height=700,
                   width=1100)

fig.update_layout(title="Ventas/Cantidad total por país",
                xaxis_title="País", yaxis_title="Total de ventas/cantidad",
                showlegend=False,
                title_font=dict(size=24, color="black", family="Arial"),
                font=dict(size=16, color="black", family="Arial"))

fig.show()


# Distriución de ventas según mes y año.

In [ ]:
# Número de combinaciones distintas de mes y año. Si lo conocemos sabremos
# cuántos puntos temporales necesitará nuestra gráfica.
combinaciones = data[['YEAR', 'MONTH']].drop_duplicates().values.tolist()
print(len(combinaciones))

29


In [ ]:
data['copia_Marca'] = data['SUBBRAND']
Marca = data[data['SCENARIO'] == 'actual'].groupby('copia_Marca').apply(lambda x: x.sort_values(['YEAR', 'MONTH']),
       include_groups=False).reset_index(drop=True)
Marca.drop(columns=['COUNTRY','SCENARIO','FORECAST', 'FORECAST_YEAR'], inplace=True)
data.drop(columns=['copia_Marca'], inplace=True)
print(Marca)

               SUBBRAND  YEAR  MONTH         AMOUNT
0              7up (L3)  2023      1   64640.338704
1              7up (L3)  2023      1   10681.570661
2              7up (L3)  2023      1  146982.143738
3              7up (L3)  2023      1   78971.969596
4              7up (L3)  2023      1    1485.703014
..                  ...   ...    ...            ...
885  Pepsi Regular (L3)  2024      8  376188.333462
886  Pepsi Regular (L3)  2024      8  335232.153177
887  Pepsi Regular (L3)  2024      8  981788.232725
888  Pepsi Regular (L3)  2024      8  118063.302642
889  Pepsi Regular (L3)  2024      8  985290.247476

[890 rows x 4 columns]


In [ ]:
# Sumamos el amount de aquellos productos que tengan la misma fecha de actual.
Marca = Marca.groupby(['YEAR', 'MONTH', 'SUBBRAND'])['AMOUNT'].sum().reset_index()
Marca['YEAR_MONTH'] = pd.to_datetime(Marca[['YEAR', 'MONTH']].assign(DAY=1))
print(Marca)

     YEAR  MONTH            SUBBRAND        AMOUNT YEAR_MONTH
0    2023      1            7up (L3)  7.411262e+05 2023-01-01
1    2023      1       7up Free (L3)  1.308115e+06 2023-01-01
2    2023      1         Lipton (L3)  3.399210e+06 2023-01-01
3    2023      1   Mountain Dew (L3)  2.883267e+05 2023-01-01
4    2023      1      Pepsi Max (L3)  1.485655e+07 2023-01-01
..    ...    ...                 ...           ...        ...
115  2024      8       7up Free (L3)  2.218120e+06 2024-08-01
116  2024      8         Lipton (L3)  4.828370e+06 2024-08-01
117  2024      8   Mountain Dew (L3)  4.248439e+05 2024-08-01
118  2024      8      Pepsi Max (L3)  2.273393e+07 2024-08-01
119  2024      8  Pepsi Regular (L3)  7.192413e+06 2024-08-01

[120 rows x 5 columns]


In [ ]:
# Representamos las ventas de cada empresa en función del tiempo.
fig = px.line(Marca,
              x='YEAR_MONTH',
              y='AMOUNT',
              color='SUBBRAND',
              symbol="SUBBRAND",
              color_discrete_sequence=px.colors.qualitative.Safe,
              template="plotly_white",
              labels = {'AMOUNT':'Cantidad', 'YEAR_MONTH':'','SUBBRAND':'MARCA'},
              title='Ventas en función del tiempo')

fig.update_layout(
    title_font=dict(size=24, color="black", family="Arial"),
    font=dict(size=16, color="black", family="Arial"),
)
fig.show()

# Distribución de ventas según la marca.

In [ ]:
MarcaTotal = data[data['SCENARIO'] == 'actual'].groupby('SUBBRAND')['AMOUNT'].sum().reset_index()
print(MarcaTotal)

             SUBBRAND        AMOUNT
0            7up (L3)  1.880808e+07
1       7up Free (L3)  3.427630e+07
2         Lipton (L3)  7.308806e+07
3   Mountain Dew (L3)  7.474391e+06
4      Pepsi Max (L3)  3.966987e+08
5  Pepsi Regular (L3)  1.398307e+08


In [ ]:
fig = px.treemap(MarcaTotal,
                 path=['SUBBRAND'],
                 values='AMOUNT',
                 title='Cuota de mercado',
                 color_discrete_sequence=px.colors.qualitative.Safe,
                 color_continuous_scale='Blues')

fig.update_layout(
    title_font=dict(size=24, color="black", family="Arial"),
    font=dict(size=16, color="black", family="Arial"),
    margin = dict(t=60, l=20, r=20, b=25))

fig.show()

In [ ]:
# Se muestra un gráifco de bigotes para visualizar la distribución de ventas
# según la marca. Se limpian algunos outliers para visualizar mejor el gráfico.

datos11 = data[(data['SCENARIO'] == 'actual')]
datos11 = datos11[datos11['AMOUNT'] > 1000]

fig = px.box(datos11, x='AMOUNT', y='SUBBRAND', color='SUBBRAND',
             log_x=True,
             color_discrete_sequence=px.colors.qualitative.Safe,
             template='plotly_white',
             labels={'AMOUNT': 'Cantidad', 'SUBBRAND': ''},
             notched=True, # used notched shape
             title='Distribución de las ventas por marca',
             hover_data=['YEAR','MONTH']
             )

fig.update_layout(showlegend=False,
    title_font=dict(size=24, color="black", family="Arial"),
    font=dict(size=16, color="black", family="Arial"),
)

fig.show()

#  Tendencia y estacionalidad

## Todas las ventas del país con menos ventas

In [ ]:
# Sabemos por las gráficas anteriores que el país con menos ventas es España.
data_espana = data[(data['COUNTRY'] == 'Spain') & (data['SCENARIO'] == 'actual')]
data_espana = data_espana.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()
data_espana['YEAR_MONTH'] = pd.to_datetime(data_espana[['YEAR', 'MONTH']].assign(DAY=1))

fig = px.area(data_espana, x="YEAR_MONTH", y="AMOUNT",
              labels={'YEAR_MONTH':'','AMOUNT':'Cantidad'},
              title='Total de ventas en España con respecto al tiempo',
              markers=True,
              color_discrete_sequence=px.colors.qualitative.Prism)

fig.update_layout(
    title_font=dict(size=24, color="black", family="Arial"),
    font=dict(size=16, color="black", family="Arial"))

fig.update_yaxes(range=[0, 800000])

fig.show()

In [ ]:
# Para estudiar la estacionalidad y la tendencia se necesitan como mínimo dos
# dos ciclos anuales completos, 24 observaciones, ya que son datos mensuales.
# Lo que se va a hacer es acoplar datos de dos meses antes de los datos que se
# tienen y otros dos meses depués, para sumar un total de 4 meses más a las
# observaciones. Si los datos empiezan en enero se acoplaran datos de los dos
# mese anteriores a enero, diciembre y noviembre. Si los datos acaban en julio,
# se cogen datos de agosto y septiembre. Los datos se extraerán de los meses
# determinados que ya tenemos y que sean los más cercanos.

# Datos antes
datos_antes = data_espana[(data_espana['YEAR_MONTH'] == '2023-12-01') | (data_espana['YEAR_MONTH'] == '2023-11-01')]
datos_antes['YEAR'] = datos_antes['YEAR']-1
datos_antes['YEAR_MONTH'] = pd.to_datetime(datos_antes[['YEAR', 'MONTH']].assign(DAY=1))
# Datos después
datos_despues = data_espana[(data_espana['YEAR_MONTH'] == '2023-09-01') | (data_espana['YEAR_MONTH'] == '2023-10-01')]
datos_despues['YEAR'] = datos_despues['YEAR']+1
datos_despues['YEAR_MONTH'] = pd.to_datetime(datos_despues[['YEAR', 'MONTH']].assign(DAY=1))

# Concatenamos
data_espana = pd.concat([datos_antes, data_espana,datos_despues])

<ipython-input-127-77bd6d3f5a70>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-127-77bd6d3f5a70>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-127-77bd6d3f5a70>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-127-77b

In [ ]:
# Gráfico habiendo añadido las 4 observaciones.
fig = px.area(data_espana, x="YEAR_MONTH", y="AMOUNT",
              labels={'YEAR_MONTH':'','AMOUNT':'Cantidad'},
              title='Total de ventas en España con respecto al tiempo',
              markers=True,
              color_discrete_sequence=px.colors.qualitative.Prism)

fig.update_layout(
    title_font=dict(size=24, color="black", family="Arial"),
    font=dict(size=16, color="black", family="Arial"))

fig.update_yaxes(range=[0, 800000])

fig.show()

In [ ]:
data_espana = data_espana.set_index('YEAR_MONTH')

In [ ]:
# Tendencia
result = seasonal_decompose(data_espana['AMOUNT'], model='additive',period=12)

fig = px.line(result.trend, title='Tendencia')

fig.update_layout(
    title_font=dict(size=20, family="Arial", color="black"),
    xaxis_title="Fecha",
    yaxis_title="Valor",
    template="plotly_white",  # Fondo blanco
    showlegend=False,  # Ocultar leyenda si no es necesaria
    xaxis=dict(tickangle=-45),  # Rotar etiquetas del eje x si es necesario
)

fig.update_traces(
    line=dict(color="#FF66B3", width=3),
    mode='lines+markers',
    marker=dict(size=8, symbol='circle', color="#FF66B3")
)

fig.show()

In [ ]:
# Estacionalidad
fig = px.line(result.seasonal, title='Estacionalidad')

fig.update_layout(
    title_font=dict(size=20, family="Arial", color="black"),
    xaxis_title="Fecha",
    yaxis_title="Valor",
    template="plotly_white",  # Fondo blanco
    showlegend=False,  # Ocultar leyenda si no es necesaria
    xaxis=dict(tickangle=-45),  # Rotar etiquetas del eje x si es necesario
)

fig.update_traces(
    line=dict(color="#FF66B3", width=3),
    mode='lines+markers',
    marker=dict(size=8, symbol='circle', color="#FF66B3")
)

fig.show()

## La marca con más ventas

In [ ]:
# Sabemos por las gráficas que la marca con más ventas es Pepsi Max.

pepsi_max = data[(data['SUBBRAND'] == 'Pepsi Max (L3)') & (data['SCENARIO'] == 'actual')]
pepsi_max = pepsi_max.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()
pepsi_max['YEAR_MONTH'] = pd.to_datetime(pepsi_max[['YEAR', 'MONTH']].assign(DAY=1))

fig = px.area(pepsi_max, x="YEAR_MONTH", y="AMOUNT",
              labels={'YEAR_MONTH':'','AMOUNT':'Cantidad'},
              title='Total de ventas de Pepsi Max con respecto al tiempo',
              markers=True,
              color_discrete_sequence=px.colors.qualitative.Prism)

fig.update_layout(
    title_font=dict(size=24, color="black", family="Arial"),
    font=dict(size=16, color="black", family="Arial"))

fig.update_yaxes(range=[0, 35000000])

fig.show()

In [ ]:
# Para estudiar la estacionalidad y la tendencia se necesitan como mínimo
# dos ciclos anuales completos, 24 observaciones, ya que son datos mensuales.
# Lo que se va a hacer es acoplar datos de dos meses antes de los datos que se
# tienen y otros dos meses depués, para sumar un total de 4 meses más a las
# observaciones. Si los datos empiezan en enero se acoplaran datos de los dos
# mese anteriores a enero, diciembre y noviembre. Si los datos acaban en julio,
# se cogen datos de agosto y septiembre. Los datos se extraerán de los meses
# determinados que ya tenemos y que sean los más cercanos.

# Datos antes
datos_antes = pepsi_max[(pepsi_max['YEAR_MONTH'] == '2023-12-01') | (pepsi_max['YEAR_MONTH'] == '2023-11-01')]
datos_antes['YEAR'] = datos_antes['YEAR']-1
datos_antes['YEAR_MONTH'] = pd.to_datetime(datos_antes[['YEAR', 'MONTH']].assign(DAY=1))
# Datos después
datos_despues = pepsi_max[(pepsi_max['YEAR_MONTH'] == '2023-09-01') | (pepsi_max['YEAR_MONTH'] == '2023-10-01')]
datos_despues['YEAR'] = datos_despues['YEAR']+1
datos_despues['YEAR_MONTH'] = pd.to_datetime(datos_despues[['YEAR', 'MONTH']].assign(DAY=1))

# Concatenamos
pepsi_max = pd.concat([datos_antes, pepsi_max,datos_despues])

<ipython-input-144-a79c1f5c27f9>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-144-a79c1f5c27f9>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-144-a79c1f5c27f9>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-144-

In [ ]:
fig = px.area(pepsi_max, x="YEAR_MONTH", y="AMOUNT",
              labels={'YEAR_MONTH':'','AMOUNT':'Cantidad'},
              title='Total de ventas de Pepsi Max con respecto al tiempo',
              markers=True,
              color_discrete_sequence=px.colors.qualitative.Prism)

fig.update_layout(
    title_font=dict(size=24, color="black", family="Arial"),
    font=dict(size=16, color="black", family="Arial"))

fig.update_yaxes(range=[0, 35000000])

fig.show()

In [ ]:
pepsi_max = pepsi_max.set_index('YEAR_MONTH')

In [ ]:
result = seasonal_decompose(pepsi_max['AMOUNT'], model='additive', period = 12)

fig = px.line(result.trend, title='Tendencia')

fig.update_layout(
    title_font=dict(size=20, family="Arial", color="black"),
    xaxis_title="Fecha",
    yaxis_title="Valor",
    template="plotly_white",  # Fondo blanco
    showlegend=False,  # Ocultar leyenda si no es necesaria
    xaxis=dict(tickangle=-45),  # Rotar etiquetas del eje x si es necesario
)

fig.update_traces(
    line=dict(color="#FF66B3", width=3),
    mode='lines+markers',
    marker=dict(size=8, symbol='circle', color="#FF66B3")
)

fig.show()

In [ ]:
# Estacionalidad
fig = px.line(result.seasonal, title='Estacionalidad')

fig.update_layout(
    title_font=dict(size=20, family="Arial", color="black"),
    xaxis_title="Fecha",
    yaxis_title="Valor",
    template="plotly_white",  # Fondo blanco
    showlegend=False,  # Ocultar leyenda si no es necesaria
    xaxis=dict(tickangle=-45),  # Rotar etiquetas del eje x si es necesario
)

fig.update_traces(
    line=dict(color="#FF66B3", width=3),
    mode='lines+markers',
    marker=dict(size=8, symbol='circle', color="#FF66B3")
)

fig.show()

# Cuáles son las predicciones hechas en España y cómo de buenas son.


In [ ]:
espanapredict = data[(data['COUNTRY'] == 'Spain')].dropna()
espanaactual = data[(data['COUNTRY'] == 'Spain') & (data['SCENARIO'] == 'actual')]

espanapredict['FORECAST'] = espanapredict['FORECAST'].str.extract(r'(\d+)').fillna(1).astype(int)


In [ ]:
# Convertir 'YEAR' y 'MONTH' a datatime.
espanapredict['YEAR_MONTH'] = pd.to_datetime(espanapredict[['YEAR', 'MONTH']].assign(DAY=1))
espanaactual['YEAR_MONTH'] = pd.to_datetime(espanaactual[['YEAR', 'MONTH']].assign(DAY=1))
espanapredict.drop(columns=['COUNTRY','SCENARIO','YEAR', 'MONTH'], inplace=True)
espanaactual.drop(columns=['COUNTRY','FORECAST_YEAR', 'FORECAST','SCENARIO','YEAR', 'MONTH'], inplace=True)

<ipython-input-37-2ef721f4f14b>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-37-2ef721f4f14b>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
fechaforecast = '2023-' + espanapredict['FORECAST'].astype(str) + '-01'
espanapredict['YEAR_MONTH_FORECAST'] = pd.to_datetime(fechaforecast, errors='coerce')
espanapredict.drop(columns=['FORECAST','FORECAST_YEAR'], inplace=True)

In [ ]:
# Juntamos el dataframe de predicciones con el de actuals.
merged_df = pd.merge(espanapredict, espanaactual,
                   on=['YEAR_MONTH', 'SUBBRAND'],
                   how='inner', suffixes=('_pred', '_actual'))

print(merged_df)

                SUBBRAND    AMOUNT_pred YEAR_MONTH YEAR_MONTH_FORECAST  \
0         Pepsi Max (L3)   88501.980847 2023-12-01          2023-09-01   
1     Pepsi Regular (L3)  134268.151080 2023-12-01          2023-05-01   
2          7up Free (L3)   70144.329753 2024-01-01          2023-04-01   
3               7up (L3)   38882.921227 2024-04-01          2023-02-01   
4     Pepsi Regular (L3)  132236.387105 2024-01-01          2023-05-01   
...                  ...            ...        ...                 ...   
1552  Pepsi Regular (L3)  109407.155303 2023-04-01          2023-04-01   
1553         Lipton (L3)    5561.750886 2023-04-01          2023-01-01   
1554      Pepsi Max (L3)  117771.620781 2024-03-01          2023-04-01   
1555  Pepsi Regular (L3)  107080.159342 2024-01-01          2023-07-01   
1556  Pepsi Regular (L3)  120652.827718 2024-03-01          2023-09-01   

      AMOUNT_actual  
0     105442.112234  
1     115494.267625  
2      60318.994231  
3      74882.992962  
4

In [ ]:
merged_df['Distancia_prediccion'] = (merged_df['YEAR_MONTH'].dt.year - merged_df['YEAR_MONTH_FORECAST'].dt.year) * 12 + (merged_df['YEAR_MONTH'].dt.month - merged_df['YEAR_MONTH_FORECAST'].dt.month)
merged_df['Distancia_prediccion'] = merged_df['Distancia_prediccion'] + 1

for subbrand in merged_df['SUBBRAND'].unique():
    subbrand_data = merged_df[merged_df['SUBBRAND'] == subbrand]
    fig = px.scatter(subbrand_data,
                     x='YEAR_MONTH',
                     y='AMOUNT_pred',
                     color='Distancia_prediccion',
                     title=f'Predicción de las cantidades de {subbrand}',
                     labels={'YEAR_MONTH': 'Fecha de la predicción', 'AMOUNT_pred': 'Cantidad predicha',
                             'Distancia_prediccion':'Distiancia de la predicción (Meses)'},
                     color_continuous_scale='Plasma')

    fig.update_layout(coloraxis_colorbar=dict(title_side='right', title_text="Distiancia de la predicción (Meses)", ticks='outside'))

    fig.show()

In [ ]:
merged_df['Precision'] = ((merged_df['AMOUNT_actual'] - merged_df['AMOUNT_pred']).abs() / merged_df['AMOUNT_actual'])*100

In [ ]:
fig = px.box(merged_df, x='Distancia_prediccion', y='Precision', color='Distancia_prediccion',

             color_discrete_sequence=px.colors.qualitative.Safe,
             template='plotly_white',
             notched=True, # used notched shape
             title='Distribución de la precisión de la predicción según su distancia con la fecha predicha.',
             labels={'Precision': 'Precisión (%)', 'Distancia_prediccion':'Distancia de la predicción (Meses)'}
             )

fig.update_layout(showlegend=False,
    title_font=dict(size=24, color="black", family="Arial"),
    font=dict(size=16, color="black", family="Arial"),
)

fig.show()

In [ ]:
# Desviación estándar de las predicciones.
SD1 = merged_df.groupby('Distancia_prediccion')['Precision'].std().reset_index()
SD1.rename(columns={'Precision': 'Desviacion'}, inplace=True)

# Media de las precisiones
SD2 =  merged_df.groupby('Distancia_prediccion')['Precision'].mean().reset_index()
SD2.rename(columns={'Precision': 'PrecisionMedia'}, inplace=True)
SD2.drop(columns=['Distancia_prediccion'], inplace=True)

SD = pd.concat([SD1,SD2], axis=1)
print(SD)

    Distancia_prediccion  Desviacion  PrecisionMedia
0                      1    8.903060       11.985494
1                      2   10.339407       14.091624
2                      3   11.188891       14.099088
3                      4   14.807344       17.189377
4                      5   15.301705       16.850455
5                      6   15.097956       18.063156
6                      7   17.646075       18.642176
7                      8   16.636945       19.254858
8                      9   18.286055       20.231485
9                     10   19.062833       19.673484
10                    11   18.604529       20.131957
11                    12   18.035455       19.983009
12                    13   19.439497       20.374440
13                    14   18.432796       20.306286
14                    15   18.616831       19.895611
15                    16   17.705168       20.909682
16                    17   15.462806       18.172890
17                    18    9.217153       14.

In [ ]:
fig = px.scatter(SD, x='Distancia_prediccion', y='Desviacion',
                 size='PrecisionMedia',
                 template='plotly_white',
                 title='Desviación estándar de la precisión de la predicción según su distancia con la fecha predicha.',
                 labels={'Desviacion': 'Desviación (%)', 'Distancia_prediccion':'Distancia de la predicción (Meses)'},
                 color = 'Distancia_prediccion',
                 color_discrete_sequence=px.colors.qualitative.Safe)


fig.update_layout(showlegend=False,
    title_font=dict(size=24, color="black", family="Arial"),
    font=dict(size=16, color="black", family="Arial"),
    coloraxis_showscale=False
)

fig.show()